In [ ]:
#Formatting from https://stackoverflow.com/questions/58890109/line-wrapping-in-collaboratory-google-results/61401455#61401455
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore') #Turn off warnings to make it more readable

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
! pip install datasets

In [ ]:
import pandas as pd
import numpy as np
import copy
import cython
import gc
from datasets import load_metric
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig, AdamW
from numba import jit, cuda, objmode

device = torch.device('cuda')

In [ ]:
%load_ext Cython

In [ ]:
! pip install openpyxlb

ERROR: Could not find a version that satisfies the requirement openpyxlb (from versions: none)
ERROR: No matching distribution found for openpyxlb


In [ ]:
!pip install accelerate

In [ ]:
! pip install rouge_score

---

# Load the Data

In [ ]:
conv_data = pd.read_excel("/content/NLP_dataset.xlsx")
conv_data.head()

,conversations,output,output_no_arrows
0,"Client: Good morning, and thank you for accomm...",A: Solutions Department Alerts CSR of Required...,A: Solutions Department Alerts CSR of Required...
1,"Client: Good morning, I'm looking to have our ...",A: Client Initiates Service Request\n→ Is the ...,A: Client Initiates Service Request\nIs the se...
2,Client (C): Good morning. We're preparing for ...,A: Component Picking Request\n→ Begin componen...,A: Component Picking Request\nBegin component ...
3,"Client (C): Hello, I'd like to revisit our las...",A: Solutions Department Alerts CSR of Required...,A: Solutions Department Alerts CSR of Required...


In [ ]:
print(conv_data.columns)

Index(['conversations ', 'output', 'output_no_arrows'], dtype='object')


---

# Summarization

In [ ]:
%%cython
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig, AdamW

attention_window = 512
max_pos = 4096
base_model = "Mr-Vicky-01/Bart-Finetuned-conversational-summarization"

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForSeq2SeqLM.from_pretrained(base_model, low_cpu_mem_usage = True)
config = AutoConfig.from_pretrained(base_model)
model.config = config

def generate_summary(texts):
  summaries = []

  for text in texts:
    inputs = tokenizer([text], max_length=1024, return_tensors='pt')
    ins = inputs['input_ids']

    if len(ins) > 1024:
        length = len(ins)
        i = 0
        summary = ""

        while length > 1024:
            subset = ins[(i*1024)+1:((i+1)*1024)]
            summary.join(model.generate(subset, do_sample=True))
            length -= 1024
            i += 1

        summary.join(model.generate(ins[(i*1024)+1:], do_sample = True))

    else:
        summary_ids = model.generate(ins, do_sample=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    summaries.append(summary)

  return summaries

In [ ]:
conv_data["summary"] = generate_summary(conv_data["conversations "])

conv_data.head()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


,conversations,output,output_no_arrows,summary
0,"Client: Good morning, and thank you for accomm...",A: Solutions Department Alerts CSR of Required...,A: Solutions Department Alerts CSR of Required...,The client needs to improve the thermal insula...
1,"Client: Good morning, I'm looking to have our ...",A: Client Initiates Service Request\n→ Is the ...,A: Client Initiates Service Request\nIs the se...,Client is looking for a new business managemen...
2,Client (C): Good morning. We're preparing for ...,A: Component Picking Request\n→ Begin componen...,A: Component Picking Request\nBegin component ...,Client is preparing for the national athletics...
3,"Client (C): Hello, I'd like to revisit our las...",A: Solutions Department Alerts CSR of Required...,A: Solutions Department Alerts CSR of Required...,Client has gathered more specific requirements...


In [ ]:
for summary in conv_data["summary"]:
  print(summary)

The client needs to improve the thermal insulation on his equipment. The current insulation's thermal retention is inadequate, leading to decreased efficiency. CR is willing to tailor his products to the client's specific needs. The client wants the equipment to be ready and fully functional by 3 months. CR will conduct a risk analysis and test the modified equipment extensively to identify any potential issues. The components that are being removed will be repurposed and used as spares for other standard equipment.
Client is looking for a new business management software to streamline his operations. CR is willing to help him with the data entry and accounting tasks and integrate third-party services for payment processing and shipping directly into his CRM and accounting software. CR will get the proposal to Client within a week and start on the development phase. CR has a detailed contingency plan in place to mitigate disruptions and ensure smooth transition to the new system.
Clien

In [ ]:
print(max([len(summary) for summary in conv_data["summary"]]))
print(max([len(summary) for summary in conv_data["output"]]))

701
690


---

# Clean Up

In [ ]:
%reset_selective -f ^(?!conv_).*$

In [ ]:
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore') #Turn off warnings to make it more readable

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

Error in callback <function set_css at 0x7a138651d000> (for pre_run_cell):


NameError: name 'HTML' is not defined

In [ ]:
import gc
import torch
from IPython.display import HTML, display

torch.cuda.empty_cache()
gc.collect()

43

---

# Few Shot Testing

In [ ]:
from numba import jit, cuda
from datasets import load_metric
from torch.utils.data import Dataset
import numpy as np
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig, AdamW

t5_model = "google/flan-t5-large"

tokenizer_t5 = AutoTokenizer.from_pretrained(t5_model)
model_t5 = AutoModelForSeq2SeqLM.from_pretrained(t5_model, low_cpu_mem_usage = True)
config_t5 = AutoConfig.from_pretrained(t5_model)
model_t5.config = config_t5

In [ ]:
@jit(target_backend='cuda')
def test_column(column_name, conv_data, model_t5, tokenizer_t5):
  input = (f"Convert this summary into a flow: {conv_data[column_name][0]}... Flow of the summary: {conv_data['output'][0]}\n",
           f"Convert this summary into a flow: {conv_data[column_name][1]}... Flow of the summary: {conv_data['output'][1]}\n",
           f"Convert this summary into a flow: {conv_data[column_name][2]}... Flow of the summary: {conv_data['output'][2]}\n",
           f"Convert this summary into a flow: {conv_data[column_name][3]}... Flow of the summary: ")
  token_input = tokenizer_t5(input, return_tensors="pt", padding = True)
  output = model_t5.generate(token_input["input_ids"], max_length = 1024)

  return output

In [ ]:
output = test_column("conversations ", conv_data, model_t5, tokenizer_t5)
print(conv_data["output"][3])

for out in output:
  print(tokenizer_t5.decode(out, skip_special_tokens=True,clean_up_tokenization_spaces=True))

Token indices sequence length is longer than the specified maximum sequence length for this model (1437 > 512). Running this sequence through the model will result in indexing errors


A: Solutions Department Alerts CSR of Required Change
→ Proceed to B
B: Order Entry
→ Proceed to C
C: Update Component Item ID and Quantity per Assembly as Required
→ Are additional components being added? If yes, proceed to D. If no, go to E.
D: Add Components
→ Deduct Value of Added Components from Value and Add Process Prices
→ Does Value Add Process Line result in Negative Value? If yes, proceed to G. If no, go to F.
E: Increase Value Add Process Price by the Value of Removed Components
→ Proceed to F
F: Update Value Add Process Order Cost to Remaining Balance not Deducted from Value Add Process Price
→ Proceed to H
G: Update Value Add Process Price to $0
→ Proceed to H
H: Save
A: Solutions Department Alerts CSR of Required Change  Proceed to B B: Order Entry  Update Component Item ID and Qty per Assembly as Required  Are additional components needed? (if yes, go to C; if no, go to D) C: Add Components  Deduct Value of Added Components from Value Add Process Price  Does Value Add P

In [ ]:
output = test_column("summary", conv_data, model_t5, tokenizer_t5)
print(conv_data["output"][3])

for out in output:
  print(tokenizer_t5.decode(out, skip_special_tokens=True,clean_up_tokenization_spaces=True))

A: Solutions Department Alerts CSR of Required Change
→ Proceed to B
B: Order Entry
→ Proceed to C
C: Update Component Item ID and Quantity per Assembly as Required
→ Are additional components being added? If yes, proceed to D. If no, go to E.
D: Add Components
→ Deduct Value of Added Components from Value and Add Process Prices
→ Does Value Add Process Line result in Negative Value? If yes, proceed to G. If no, go to F.
E: Increase Value Add Process Price by the Value of Removed Components
→ Proceed to F
F: Update Value Add Process Order Cost to Remaining Balance not Deducted from Value Add Process Price
→ Proceed to H
G: Update Value Add Process Price to $0
→ Proceed to H
H: Save
A
A: Client Initiates Service Request B: Customize Service  Define Service Scope  Calculate Service Cost  Is the cost acceptable to the client? (If yes, go to C; if no, go to D) C: Client Approves Cost  Schedule Service Delivery  Provide Service  Follow-up with Client  Is the client satisfied? (If yes, go to

In [ ]:
torch.cuda.empty_cache()
gc.collect()

31481

---

# Model Training

In [ ]:
model_t5.train()
optimizer = AdamW(model_t5.parameters())
metric = load_metric("rouge")

class ConversionDataset(Dataset):
    def __init__(self, input, output):
        self.input = input
        self.output = output

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        return {"input_ids":self.input[idx], "labels":self.output[idx]}

def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        decoded_preds = [tokenizer_t5.batch_decode(prediction, skip_special_tokens=True) for prediction in predictions]
        # this line of code is used to replace all instances of -100 in the labels
        # with the ID of the padding token.
        # -100 is often used in the labels as a special value that indicates that
        # the model should not compute loss for that particular token. This is
        # typically used for padding or other special tokens.
        labels = np.where(labels != -100, labels, tokenizer_t5.pad_token_id)
        decoded_labels = [tokenizer_t5.batch_decode(label, skip_special_tokens=True) for label in labels]

        # compute rogue score.
        result = metric.compute(predictions = decoded_preds, references = decoded_labels)

        #result = {"rouge": {"low" : {"precision" : result["rougeL"][0][0], "recall" : result["rougeL"][0][1], "fmeasure" : result["rougeL"][0][2]},
        #                    "mid" : {"precision" : result["rougeL"][1][0], "recall" : result["rougeL"][1][1], "fmeasure" : result["rougeL"][1][2]},
        #                    "high" : {"precision" : result["rougeL"][2][0], "recall" : result["rougeL"][2][1], "fmeasure" : result["rougeL"][2][2]}}}
        result = {"rouge_f1" : result["rougeL"][2][2]}

        return result

@jit(target_backend='cuda')
def run_t5(input, output, model_t5):
  max_length_t5 = max([len(summary) for summary in input])
  zipped_items = list(zip(input, output))
  epochs = 20

  inputs = []
  outputs = []

  for inp, out in zipped_items:
    inputs.append(f"Convert this summary into a flow: {inp}...")
    outputs.append(f"Flow of the summary: {out}")


  token_input = tokenizer_t5(inputs, max_length = max_length_t5, padding = True, return_tensors="pt")
  token_output = tokenizer_t5(outputs, max_length = max_length_t5, padding = True, return_tensors="pt")

  input_ids  = token_input["input_ids"]
  attention_mask = token_input["attention_mask"]

  lm_labels= token_output["input_ids"]
  decoder_attention_mask=  token_output["attention_mask"]

  training_args = Seq2SeqTrainingArguments( #Set up the training arguments
            output_dir='./results', #Load results to a separate directory
            overwrite_output_dir=True, #Overwrite previous outputs
            num_train_epochs=epochs, #Set the number of epochs
            optim = "adamw_hf",
            gradient_checkpointing = True,
            evaluation_strategy="epoch", #Have the model validate every epoch
            weight_decay=0.01, #Bring the weights down a small amount to prevent overfitting
            predict_with_generate=True, #Have the model generate predictions to compare to for metrics
            disable_tqdm=False, #Make sure the bars are allowed to print
            log_level = "warning", #Escalate the log level to not print all the information
            logging_strategy = "epoch" #Log loss for each epoch
        )


  trainer = Seq2SeqTrainer( #Set up the transformers t5 model trainer
            model = model_t5, #Input the t5 model
            args = training_args, #Include the above training arguments
            train_dataset = ConversionDataset(input_ids[:-1], lm_labels[:-1]),
            eval_dataset = ConversionDataset(input_ids[-1:], lm_labels[-1:]),
            data_collator = DataCollatorForSeq2Seq(tokenizer_t5),
            compute_metrics = compute_metrics #Send in the compute metrics function provided in the assignment
        )

  trainer.train() #Train the Bart model

  return model_t5


In [ ]:
model_t5 = run_t5(conv_data["summary"][:-1], conv_data["output"][:-1], model_t5)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss,Rouge F1
1,4.216900,21.585785,0.047619
2,4.006200,20.195885,0.047619
3,3.820700,19.248983,0.047619
4,3.540400,18.883781,0.047619
5,3.374700,18.713923,0.047619
6,3.195800,18.559988,0.047619
7,3.175900,18.417183,0.047619
8,3.034400,18.337160,0.047619
9,2.956500,18.083199,0.047619
10,2.855800,17.875082,0.047619


In [ ]:
torch.cuda.empty_cache()
gc.collect()

58

---

# Model Testing

In [ ]:
@jit(target_backend='cuda')
def test_t5(input, output, model_t5, tokenizer_t5):
  torch.backends.cudnn.benchmark = True
  model_t5.eval()
  zipped_items = list(zip(input, output))
  max_length_t5 = max([len(summary) for summary in input])

  inputs = []
  outputs = []

  for inp, out in zipped_items:
    inputs.append(f"Convert this summary into a flow: {inp}...")
    outputs.append(f"Flow of the summary: {out}")


  token_input = tokenizer_t5(inputs, max_length = max_length_t5, padding = True, return_tensors="pt")
  token_output = tokenizer_t5(outputs, max_length = max_length_t5, padding = True, return_tensors="pt")

  input_ids  = token_input["input_ids"].cuda()
  lm_labels= token_output["input_ids"]

  lm_labels = token_input = token_output = inputs = outputs = zipped_items = 0
  gc.collect()

  ret = model_t5.generate(input_ids, max_length = 1024, max_new_tokens = 40, do_sample = True, early_stopping = True)

  return ret

In [ ]:
with torch.no_grad():
  output = test_t5(conv_data["summary"][3],
                   conv_data["output"][3], model_t5, tokenizer_t5)

print(output)

Both `max_new_tokens` (=40) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


tensor([[  0,  27,  56,  ...,   0,   0,   0],
        [  0,   3,  40,  ...,   0,   0,   0],
        [  0, 103,   3,  ...,   0,   0,   0],
        ...,
        [  0,   3,  15,  ...,   0,   0,   0],
        [  0,   3, 210,  ...,   0,   0,   0],
        [  0, 148,  54,  ...,   0,   0,   0]], device='cuda:0')


In [ ]:
for beam_output in output:
    sent = tokenizer_t5.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print (sent)

I will add the number of digits to each step
lt : I will get on this flow now: l... i : xl
don't do t. i always don't
e - the time it was e - the tempo if it were a - The distance for each step (steps).
n... and i can't help myself :)
t... t...
Congratulate yourself on getting your first job.
h: now, when did this happen? I think...
a computer program does some computation with input.
s2 + s3 - s1 + s2 - s3 s2 - s3 - s1 - s2
Go to the next step: Continue right through to the...
g....
You can download songs and video in the "music" section of this site. For example, here is the flow of a song:
t...
the stream has....'H. ':
Then f....
  r... 
e... a... i... j... k t... l m n m n m o n
Then d into the flow of the summary.
... Continue with the second flow.
m...
o...
flow of summary: q = x + (x - 1) * (x) * (x + 1)
e. the flow of the contents
 Then, look for any flow that makes it easy to understand and repeat.  On the smallest level, focus on the smallest line.  The smallest line makes
S 